In [1]:
from config import *
import cv2
import numpy as np
from config import IMAGE_WIDTH, IMAGE_HEIGHT
from carla_utils.carla_manager import CarlaManager
from carla_utils.sensor_manager import SensorManager
from yolo_utils.yolo_manager import YoloManager
from utils.helpers import process_image

In [2]:
 # Initialize managers
carla_manager = CarlaManager()

# Change the map
carla_manager.world = carla_manager.client.load_world(CARLA_MAP)
vehicle = carla_manager.spawn_vehicle()

sensor_manager = SensorManager(carla_manager.world, vehicle)
yolo_manager = YoloManager()


In [3]:
#define environment/world and get possible places to spawn a car
world = client.get_world()
print(world.get_map().name)
spawn_points = world.get_map().get_spawn_points()
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)






#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') 
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))

#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

Town03


In [4]:
import sys

# Add CARLA PythonAPI to the system path
sys.path.append('C:\carla\PythonAPI\carla')

# Import GlobalRoutePlanner
from agents.navigation.global_route_planner import GlobalRoutePlanner


# Set up the GlobalRoutePlanner
#sampling_resolution = 1.0  # Distance between waypoints (in meters)
#grp = GlobalRoutePlanner(world.get_map(), sampling_resolution)

grp = GlobalRoutePlanner(world.get_map())
#sampling_resolution = 1.0
#grp.setup(sampling_resolution)

# Define the starting point (e.g., the car's current location)
start_point = spawn_points[0]  # Replace with the actual starting point
point_a = start_point.location

# Find the longest route
distance = 0
route = None
for loc in spawn_points:
    try:
        cur_route = grp.trace_route(point_a, loc.location)
        if len(cur_route) > distance:
            distance = len(cur_route)
            route = cur_route
    except Exception as e:
        print(f"Error computing route: {e}")
        continue

# Visualize the route
if route:
    for i in range(len(route) - 1):
        world.debug.draw_line(
            route[i][0].transform.location, route[i + 1][0].transform.location,
            thickness=0.1, color=carla.Color(r=0, g=255, b=0), life_time=60.0
        )
    print(f"Longest route found with {distance} waypoints.")
else:
    print("No valid route found.")






#draw the route in sim window - Note it does not get into the camera of the car
#for waypoint in route:
 #   world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
  #      color=carla.Color(r=0, g=0, b=255), life_time=60.0,
   #     persistent_lines=True)

TypeError: setup() takes 1 positional argument but 2 were given

In [ ]:
# the cheating loop of moving the car along the route
for waypoint in route:
    
    # move the car to current waypoint
    vehicle.set_transform(waypoint[0].transform)
    # Dispaly with imshow
    cv2.imshow('Fake self-driving',camera_data['image'])
    cv2.waitKey(50)
    
time.sleep(2)
cv2.destroyAllWindows()
camera.stop() # this is the opposite of camera.listen
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()